In [ ]:
#| default_exp kernel

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

# ez_kaggle.kernel

> API details for managing notebooks as kaggle kernels

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import string, ast, json
from ez_kaggle.setup import *
from nbdev.processors import nb_fmdict
from execnb.nbio import read_nb
from pathlib import Path

In [ ]:
#|export
def nb2names(file):
    nb = read_nb(file)
    title = nb_fmdict(nb)['title']
    bad_chars = '|'.join(string.punctuation+string.whitespace)
    translator = title.maketrans(bad_chars,'-'*len(bad_chars))
    id = title.translate(translator)
    return title, id.lower()

In [ ]:
nb2names('index.ipynb')

('ez_kaggle', 'ez-kaggle')

If you pass a list of space separated modules to `install`, they'll be installed if running on Kaggle.

In [ ]:
#|export
def nb_meta(file, 
            private=False, 
            gpu=False, 
            internet=True,                              
            language='python',
            cfg_path='.',
            **kwargs # Config dict to overwrite or replace fastkaggle.json
           ):
    "Get the `dict` required for a kernel-metadata.json file"    
    cfg = get_config_values(cfg_path,**kwargs)
    competition = cfg['competition']
    cfg_datasets = [f"{cfg['datasets_username']}/{cfg['model_dataset_name']}",
                    f"{cfg['datasets_username']}/{cfg['libraries_dataset_name']}"]
    title,id = nb2names(file)    

    d = {
      "id": f"{get_username()}/{id}",
      "title": title,
      "code_file": file,
      "language": language,
      "kernel_type": "notebook",
      "is_private": private,
      "enable_gpu": gpu,
      "enable_internet": internet,
      "keywords": [],
      "dataset_sources": cfg_datasets if cfg_datasets else [],
      "kernel_sources": []
    }
    if competition: d["competition_sources"] = [f"competitions/{competition}"]
    return d

In [ ]:
meta_new = nb_meta('index.ipynb')

In [ ]:
#|export
def push_notebook(file, cfg_path='.', private=False, gpu=True, internet=True, **kwargs):
    "Push notebook `file` to Kaggle Notebooks"
    meta = nb_meta(file=file, cfg_path=cfg_path, private=private, gpu=gpu, internet=internet, **kwargs)
    path = Path(file).parent
    nm = 'kernel-metadata.json'
    path.mkdir(exist_ok=True, parents=True)
    with open(path/nm, 'w') as f: json.dump(meta, f, indent=2)    
    api = import_kaggle()
    api.kernels_push_cli(str(path))

In [ ]:
#|eval: false
push_notebook('index.ipynb')

Kernel version 3 successfully pushed.  Please check progress at https://www.kaggle.com/code/isaacflath/ez-kaggle


## Export -

In [ ]:
#|hide

from nbdev.doclinks import nbdev_export
nbdev_export()